In [ ]:
######SEGMENTATION CH02, ONLY OUTSIDE AND CELL BODY WITH LABEL ##################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion, ball

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.4*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
    colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        name='Lysosomes (cell vs outside)'
    )
    try:
        pts.face_color = colors
        pts.edge_color = 'black'
        pts.edge_width = 0.3
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        #pts.text = {'text': 'C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if len(df) > 0 and n_cells > 0 and body_lab.max() > 0:
        # cells with lysosomes inside 'cell' mask
        """
        cell_hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
        ids_with_lyso = set(np.unique(cell_hits["cell_id_ch2"].to_numpy()))
        all_ids = set(range(1, int(body_lab.max()) + 1))
        ids_without_lyso = all_ids - ids_with_lyso

        props = regionprops(body_lab)
        pts_yes, labels_yes = [], []
        pts_no, labels_no   = [], []

        for prop in props:
            cid = int(prop.label)
            cz, cy, cx = prop.centroid  # (z, y, x)
            if cid in ids_with_lyso:
                pts_yes.append([cz, cy, cx])
                labels_yes.append(f"Cell {cid}")
            else:
                pts_no.append([cz, cy, cx])
                labels_no.append(f"Cell {cid}")

        # layer: cell labels (HAS lysosomes) - yellow
        if pts_yes:
            lbl_yes = viewer.add_points(
                np.asarray(pts_yes, dtype=float),
                name="Cell labels (HAS lysosomes)",
                size=0.1
            )
            lbl_yes.face_color = [0, 0, 0, 0]
            lbl_yes.edge_color = [0, 0, 0, 0]
            lbl_yes.edge_width = 0
            lbl_yes.text = {"text": labels_yes, "size": 28, "color": "yellow", "anchor": "center"}
            lbl_yes.blending = "translucent_no_depth"

        # layer: cell labels (NO lysosomes) - light gray
        if pts_no:
            lbl_no = viewer.add_points(
                np.asarray(pts_no, dtype=float),
                name="Cell labels (NO lysosomes)",
                size=0.1
            )
            lbl_no.face_color = [0, 0, 0, 0]
            lbl_no.edge_color = [0, 0, 0, 0]
            lbl_no.edge_width = 0
            lbl_no.text = {"text": labels_no, "size": 28, "color": "lightgray", "anchor": "center"}
            lbl_no.blending = "translucent_no_depth"
        """
        # (optional) stamp IDs sparsely across territories
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)
    
# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        # top-down view: look along -Z (XY plane), roll 0
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        # older napari API fallback
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # side view: look along -X (show YZ)
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # front view: look along -Y (show XZ)
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
######################SEGMENTATION CH02, ONLY OUTSIDE AND CELL BODY WITH LABEL more volumen ch_02_cell##############

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion, ball

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=91, offset=-0.3*np.std(R))#####0.4###81
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(7))#3
neuron_mask = binary_erosion(neuron_mask, ball(1))  # light extra erosion

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# --------------------------
# 5b) Per-cell (Ch2) volumes
# --------------------------
# Try to get voxel size (µm) from CZI metadata; fallback = 1 µm
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis):
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um  # arrays are Z,Y,X

cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy()
            }
            #pts.text = {'text': 'C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and body_lab.max() > 0:
        # (optional) stamp IDs sparsely across territories
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        # top-down view: look along -Z (XY plane), roll 0
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        # older napari API fallback
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # side view: look along -X (show YZ)
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # front view: look along -Y (show XZ)
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
##############################SEGMENTATION CH02, ONLY OUTSIDE AND CELL BODY WITH LABEL more volumen ch_02_cell and >205000##############

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=81, offset=-0.3*np.std(R))#0.4
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(3))

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# --------------------------
# 4b) Remove oversized territories (avoid in cell_id_ch2)
# --------------------------
#MAX_CELL_VOXELS = 205000  # threshold to exclude huge territories
MAX_CELL_VOXELS = 210000
if cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    big_ids = np.flatnonzero(counts > MAX_CELL_VOXELS)
    big_ids = big_ids[big_ids != 0]  # never treat background as "oversized"
    if big_ids.size > 0:
        big_mask = np.isin(cell_seg, big_ids)
        cell_seg[big_mask] = 0
        cell_mask[big_mask] = False
        print(f"Removed {big_ids.size} oversized territories (> {MAX_CELL_VOXELS} voxels): {big_ids.tolist()}")

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if cell_seg[zz, yy, xx] > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# --------------------------
# 5b) Per-cell (Ch2) volumes
# --------------------------
# Try to get voxel size (µm) from CZI metadata; fallback = 1 µm
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis):
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um  # arrays are Z,Y,X

cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if cell_seg.max() > 0:
    counts_after = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts_after.size, dtype=int)
    voxels = counts_after[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy()
            }
            #pts.text = {'text': 'C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if cell_seg.max() > 0:
        # (optional) stamp IDs sparsely across territories
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
######SEGMENTATION CH02, ONLY OUTSIDE AND CELL BODY WITH LABEL AND CONVERTION A um###################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries

# --------------------------
# 0) Tunables
# --------------------------
# Radius threshold for "cell" in microns (use your biological prior; adjust as needed)
cell_min_radius_um = 1.0

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

def _get_voxel_size_um_from_metadata(mdxml: str):
    """Parse CZI metadata XML to (dz, dy, dx) in microns. Falls back to (1,1,1)."""
    try:
        root = ET.fromstring(mdxml)
        def val_um(axis_id):
            v_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/Value')
            u_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/DefaultUnit')
            if v_node is None:
                return None
            v = float(v_node.text)
            unit = (u_node.text if u_node is not None else 'm').lower()
            if unit in ('m', 'meter', 'metre'):
                return v * 1e6
            if unit in ('µm', 'um', 'micrometer', 'micrometre'):
                return v
            # unknown -> assume meters
            return v * 1e6

        dx = val_um('X')
        dy = val_um('Y')
        dz = val_um('Z')
        if dx and dy and dz:
            return float(dz), float(dy), float(dx)
    except Exception as e:
        print("Voxel size parse failed:", e)
    return 1.0, 1.0, 1.0

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    mdxml = czi.metadata()  # XML string

voxel_um = _get_voxel_size_um_from_metadata(mdxml)
print(f"Voxel size (µm): dz={voxel_um[0]:.4f}, dy={voxel_um[1]:.4f}, dx={voxel_um[2]:.4f}")

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins (optional export kept)
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes,
    "region_id": region_labels
})

if len(df) > 0:
    region_summary = df.groupby("region_id").agg(
        count=("radius_voxels", "size"),
        avg_diameter=("diameter_voxels", "mean"),
        total_volume=("volume_voxels", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=["region_id", "count", "avg_diameter", "total_volume"])

df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE) in microns
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=91, offset=-0.3*np.std(R))#####0.4###81
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(7))#3
neuron_mask = binary_erosion(neuron_mask, ball(1))  # light extra erosion

# Distance transform **in microns** using anisotropic sampling
dist_um = edt(neuron_mask, sampling=voxel_um)  # units: microns
# CELL = points whose radius to background >= cell_min_radius_um
cell_mask = (dist_um >= cell_min_radius_um)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    # watershed on **foreground**, still fine; territories used only for per-cell IDs
    dist_inside_um = edt(neuron_mask, sampling=voxel_um)
    cell_seg = watershed(-dist_inside_um, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print(f"cell voxels (>= {cell_min_radius_um} µm):", int(cell_mask.sum()))

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist())
    colors = np.zeros((len(loc), 4), dtype=float)
    colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
    colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        name='Lysosomes (cell vs outside)'
    )
    try:
        pts.face_color = colors
        pts.edge_color = 'black'
        pts.edge_width = 0.3
        pts.properties = {
            'lys_id': df['id'].to_numpy(),
            'where':  df['location_ch2'].to_numpy(),
            'cell':   df['cell_id_ch2'].to_numpy()
        }
        #pts.text = {'text': 'C:{cell} {where}', 'size': 8, 'color': 'white', 'anchor': 'upper left'}
        pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if len(df) > 0 and n_cells > 0 and body_lab.max() > 0:
        # cells with lysosomes inside 'cell' mask
        """
        cell_hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
        ids_with_lyso = set(np.unique(cell_hits["cell_id_ch2"].to_numpy()))
        all_ids = set(range(1, int(body_lab.max()) + 1))
        ids_without_lyso = all_ids - ids_with_lyso

        props = regionprops(body_lab)
        pts_yes, labels_yes = [], []
        pts_no, labels_no   = [], []

        for prop in props:
            cid = int(prop.label)
            cz, cy, cx = prop.centroid  # (z, y, x)
            if cid in ids_with_lyso:
                pts_yes.append([cz, cy, cx])
                labels_yes.append(f"Cell {cid}")
            else:
                pts_no.append([cz, cy, cx])
                labels_no.append(f"Cell {cid}")

        # layer: cell labels (HAS lysosomes) - yellow
        if pts_yes:
            lbl_yes = viewer.add_points(
                np.asarray(pts_yes, dtype=float),
                name="Cell labels (HAS lysosomes)",
                size=0.1
            )
            lbl_yes.face_color = [0, 0, 0, 0]
            lbl_yes.edge_color = [0, 0, 0, 0]
            lbl_yes.edge_width = 0
            lbl_yes.text = {"text": labels_yes, "size": 28, "color": "yellow", "anchor": "center"}
            lbl_yes.blending = "translucent_no_depth"

        # layer: cell labels (NO lysosomes) - light gray
        if pts_no:
            lbl_no = viewer.add_points(
                np.asarray(pts_no, dtype=float),
                name="Cell labels (NO lysosomes)",
                size=0.1
            )
            lbl_no.face_color = [0, 0, 0, 0]
            lbl_no.edge_color = [0, 0, 0, 0]
            lbl_no.edge_width = 0
            lbl_no.text = {"text": labels_no, "size": 28, "color": "lightgray", "anchor": "center"}
            lbl_no.blending = "translucent_no_depth"
        """
        # (optional) stamp IDs sparsely across territories
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        # top-down view: look along -Z (XY plane), roll 0
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        # older napari API fallback
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # side view: look along -X (show YZ)
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # front view: look along -Y (show XZ)
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
###############LAST VERSION#################################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion  # no second 'ball' import

# --------------------------
# 0) Tunables
# --------------------------
# Radius threshold for "cell" in microns (use your biological prior; adjust as needed)
cell_min_radius_um = 1.0

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

def _get_voxel_size_um_from_metadata(mdxml: str):
    """Parse CZI metadata XML to (dz, dy, dx) in microns. Falls back to (1,1,1)."""
    try:
        root = ET.fromstring(mdxml)
        def val_um(axis_id):
            v_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/Value')
            u_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/DefaultUnit')
            if v_node is None:
                return None
            v = float(v_node.text)
            unit = (u_node.text if u_node is not None else 'm').lower()
            if unit in ('m', 'meter', 'metre'):
                return v * 1e6
            if unit in ('µm', 'um', 'micrometer', 'micrometre'):
                return v
            # unknown -> assume meters
            return v * 1e6

        dx = val_um('X')
        dy = val_um('Y')
        dz = val_um('Z')
        if dx and dy and dz:
            return float(dz), float(dy), float(dx)
    except Exception as e:
        print("Voxel size parse failed:", e)
    return 1.0, 1.0, 1.0

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    mdxml = czi.metadata()  # XML string

voxel_um = _get_voxel_size_um_from_metadata(mdxml)
print(f"Voxel size (µm): dz={voxel_um[0]:.4f}, dy={voxel_um[1]:.4f}, dx={voxel_um[2]:.4f}")

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob DF & region bins
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)
volumes_vox = (4/3) * np.pi * (blobs[:, 3]**3) if len(blobs) > 0 else np.array([])

df = pd.DataFrame({
    "id": blob_ids,
    "z": blobs[:, 0] if len(blobs) > 0 else [],
    "y": blobs[:, 1] if len(blobs) > 0 else [],
    "x": blobs[:, 2] if len(blobs) > 0 else [],
    "diameter_voxels": blobs[:, 3] * 2 if len(blobs) > 0 else [],
    "radius_voxels": blobs[:, 3] if len(blobs) > 0 else [],
    "volume_voxels": volumes_vox,
    "region_id": region_labels
})

# --- Add pixel→µm conversions to the lysosome table ---
if len(df) > 0:
    # Coordinates in µm (origin at voxel index 0)
    df["z_um"] = df["z"] * vz_um
    df["y_um"] = df["y"] * vy_um
    df["x_um"] = df["x"] * vx_um

    # Sizes in µm (note anisotropy: XY vs Z)
    df["radius_um_xy"]    = df["radius_voxels"] * vx_um
    df["radius_um_z"]     = df["radius_voxels"] * vz_um
    df["diameter_um_xy"]  = df["diameter_voxels"] * vx_um
    df["diameter_um_z"]   = df["diameter_voxels"] * vz_um

    # Volumes in µm³: two flavors
    # 1) iso: voxel-sphere * voxel_um3 (exact if cubic voxels)
    df["volume_um3_iso"]  = df["volume_voxels"] * voxel_um3
    # 2) xyz: ellipsoid using anisotropic radii (more faithful with non-cubic voxels)
    df["volume_um3_xyz"]  = (4.0/3.0) * np.pi * (df["radius_um_xy"]**2) * df["radius_um_z"]

    # Region summary with µm columns
    region_summary = df.groupby("region_id").agg(
        count=("id", "size"),
        avg_diameter_vox=("diameter_voxels", "mean"),
        avg_diameter_um_xy=("diameter_um_xy", "mean"),
        avg_diameter_um_z=("diameter_um_z", "mean"),
        total_volume_vox=("volume_voxels", "sum"),
        total_volume_um3_iso=("volume_um3_iso", "sum"),
        total_volume_um3_xyz=("volume_um3_xyz", "sum"),
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=[
        "region_id","count","avg_diameter_vox","avg_diameter_um_xy","avg_diameter_um_z",
        "total_volume_vox","total_volume_um3_iso","total_volume_um3_xyz"
    ])

# Save early lysosome-region summaries (now include µm fields)
df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=91, offset=-0.3*np.std(R))  # tuned
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(7))
neuron_mask = binary_erosion(neuron_mask, ball(1))  # extra light erosion

# CELL (soma) via distance transform
dist = edt(neuron_mask)
cell_min_radius_vox = 1   # tune (e.g., 4–6) for thicker soma
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# --------------------------
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# --------------------------
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table (now includes µm columns)
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# --------------------------
# 5b) Per-cell (Ch2) volumes
# --------------------------
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3,
        "vx_um": vx_um,
        "vy_um": vy_um,
        "vz_um": vz_um,
        "voxel_um3": voxel_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and body_lab.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        # top-down view: look along -Z (XY plane), roll 0
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        # older napari API fallback
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # side view: look along -X (show YZ)
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # front view: look along -Y (show XZ)
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()

In [ ]:
#####LAST VERSION 1 R,D AND V##################################3

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion  # no second 'ball' import

# --------------------------
# 0) Tunables
# --------------------------
# Radius threshold for "cell" in microns (use your biological prior; adjust as needed)
cell_min_radius_um = 1.0

# --------------------------
# 1) LOAD .CZI IMAGE
# --------------------------
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_1_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/Airy scan_40A_UAS-TMEM-HA_CB_4h_2_051222.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

def _get_voxel_size_um_from_metadata(mdxml: str):
    """Parse CZI metadata XML to (dz, dy, dx) in microns. Falls back to (1,1,1)."""
    try:
        root = ET.fromstring(mdxml)
        def val_um(axis_id):
            v_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/Value')
            u_node = root.find(f'.//Scaling/Items/Distance[@Id="{axis_id}"]/DefaultUnit')
            if v_node is None:
                return None
            v = float(v_node.text)
            unit = (u_node.text if u_node is not None else 'm').lower()
            if unit in ('m', 'meter', 'metre'):
                return v * 1e6
            if unit in ('µm', 'um', 'micrometer', 'micrometre'):
                return v
            # unknown -> assume meters
            return v * 1e6

        dx = val_um('X')
        dy = val_um('Y')
        dz = val_um('Z')
        if dx and dy and dz:
            return float(dz), float(dy), float(dx)
    except Exception as e:
        print("Voxel size parse failed:", e)
    return 1.0, 1.0, 1.0

with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    mdxml = czi.metadata()  # XML string

# --- voxel sizes & volume (µm) ---
voxel_um = _get_voxel_size_um_from_metadata(mdxml)  # (dz, dy, dx) in µm
vz_um, vy_um, vx_um = voxel_um
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): dz={vz_um:.4f}, dy={vy_um:.4f}, dx={vx_um:.4f}")

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --------------------------
# 2) CH1: blobs (lysosomes)
# --------------------------
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # sigma->radius (3D)
print(f"Detected {len(blobs)} lysosomes.")

# Per-blob region bins
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

blob_ids = np.arange(1, len(blobs) + 1)

# --- Per-lysosome metrics in micrometers only (single set) ---
if len(blobs) > 0:
    r_vox = blobs[:, 3]  # radius in voxels

    # Anisotropic ellipsoid radii in µm
    rx_um = r_vox * vx_um
    ry_um = r_vox * vy_um
    rz_um = r_vox * vz_um

    # Volume in µm^3 using ellipsoid formula
    volume_um3 = (4.0 / 3.0) * np.pi * rx_um * ry_um * rz_um

    # Equivalent-sphere radius & diameter in µm
    radius_um   = ((3.0 * volume_um3) / (4.0 * np.pi)) ** (1.0 / 3.0)
    diameter_um = 2.0 * radius_um

    df = pd.DataFrame({
        "id": blob_ids,
        "z": blobs[:, 0],
        "y": blobs[:, 1],
        "x": blobs[:, 2],
        "region_id": region_labels,
        "radius_um": radius_um,
        "diameter_um": diameter_um,
        "volume_um3": volume_um3
    })
else:
    df = pd.DataFrame(columns=[
        "id","z","y","x","region_id","radius_um","diameter_um","volume_um3"
    ])

# --- Add coords in µm (kept separate; no extra size variants) ---
if len(df) > 0:
    df["z_um"] = df["z"] * vz_um
    df["y_um"] = df["y"] * vy_um
    df["x_um"] = df["x"] * vx_um

    # Region summary with single µm metrics
    region_summary = df.groupby("region_id").agg(
        count=("id", "count"),
        avg_diameter_um=("diameter_um", "mean"),
        total_volume_um3=("volume_um3", "sum")
    ).reset_index()
else:
    region_summary = pd.DataFrame(columns=[
        "region_id","count","avg_diameter_um","total_volume_um3"
    ])

# Save early lysosome-region summaries (µm-only size metrics)
df.to_csv("lysosome_blobs_regions.csv", index=False)
region_summary.to_csv("lysosome_region_summary.csv", index=False)
print("Saved: lysosome_blobs_regions.csv and lysosome_region_summary.csv")

# --------------------------
# 3) Viewer base
# --------------------------
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# --------------------------
# 4) CH2: segmentation (CELL vs OUTSIDE)
# --------------------------
# Normalize & denoise
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=0.6, preserve_range=True)

# Base neuron "foreground" via local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=91, offset=-0.3*np.std(R))  # tuned
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(7))
neuron_mask = binary_erosion(neuron_mask, ball(1))  # extra light erosion

# CELL (soma) via distance transform
dist = edt(neuron_mask)

# Convert desired soma radius (µm) to voxels (use smallest axis conservatively)
voxel_min_um = min(vx_um, vy_um, vz_um)
cell_min_radius_vox = max(1, int(round(cell_min_radius_um / voxel_min_um)))

cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(3))
cell_mask = binary_closing(cell_mask, ball(4))

# Label all cells (for watershed seeding) and partition territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

# Sanity
print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))
# --------------------------
# 5) Map lysosomes to neuron vs outside, and assign per-cell IDs
# --------------------------
inside_mask = neuron_mask  # use full neuron territory, not just soma

location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = inside_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        in_bounds = (0 <= zz < Z) and (0 <= yy < Y) and (0 <= xx < X)
        if in_bounds and inside_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Quick sanity print
    try:
        inside_cnt = sum(df["location_ch2"] == "cell")
        print(f"Lysosomes inside neuron: {inside_cnt} / {len(df)} "
              f"({100.0 * inside_cnt / max(1, len(df)):.1f}%)")
    except Exception:
        pass

    # Per-class counts
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# --------------------------
# 5b) Per-cell (Ch2) volumes
# --------------------------
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3,
        "vx_um": vx_um,
        "vy_um": vy_um,
        "vz_um": vz_um,
        "voxel_um3": voxel_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# --------------------------
# 6) Visualization (CELL vs OUTSIDE only)
# --------------------------
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and body_lab.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES and cell_seg.max() > 0:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# --------------------------
# 7) Quick fused 2D video (optional)
# --------------------------
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# --------------------------
# 8) Save a 3D screenshot (post-segmentation)
# --------------------------
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        # top-down view: look along -Z (XY plane), roll 0
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        # older napari API fallback
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # side view: look along -X (show YZ)
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        # front view: look along -Y (show XZ)
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# --------------------------
# 9) Run viewer
# --------------------------
napari.run()